In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [31]:
N = 100_000
greater = 0
tot = 0

for _ in range(N):
    group = [np.random.uniform(0,2) for i in range(3)]
    if np.median(group) > 1.5:
        greater += 1
    tot += 1

print(greater/tot)

0.15789


In [33]:
np.median([0, 1.5,1.51])

1.5

In [34]:
0.25*0.25*0.75*3 + 0.25**3

0.15625

In [37]:
5/16

0.3125